# Parsing text with PyParsing

Tomasz Tylec, PhD<br/>
Institute of Theoretical Physics and Astrophysics, UG<br/>
and
<a href="http://zettafox.com" style="text-decoration: none"><b><font face="Book Antiqua, serif" color="#7f7f7f"><span style="font-size:120%;line-height:107%">zetta</span></font></b><b><font color="#ed7d31"><font face="Book Antiqua, serif"><span style="font-size:120%;line-height:107%">fox <br></span></font></font></b></a>

## What we want?

From [wikipedia](https://en.wikipedia.org/wiki/Parsing)

> **Parsing**, syntax analysis or syntactic analysis is the process of analysing a string of symbols, either in natural language or in computer languages, conforming to the rules of a formal grammar. 

Basically: transform string to some data structure. For example:

- extract *protocol*, *user*, *host* and *repostitory* from strings like:
  `ssh://user@server/project.git`, `https://example.com/gitproject.git`, `file:///opt/git/project.git`

- construct the AST (Abstract Syntax Tree) of some HTML code (in order to render it)

- interpret command from user typed in CLI (my use-case)

## How we do it?

We can use regular expressions:

In [1]:
import re
r = re.compile(r"(?P<proto>file://|https://|ssh://)((?P<user>[a-zA-Z0-9]+)@)?(?P<host>[a-zA-Z0-9\.]+)")

m = re.match(r, "ssh://foo@server")
{k: m[k] for k in ['proto', 'user', 'host']}

{'host': 'server', 'proto': 'ssh://', 'user': 'foo'}

Regex'es have drawbacks:
- hard to maintain;
- (true) regex cannot parse everything (e.g. valid XHTML)...
- ... although PCRE can do [pretty much everything](http://nikic.github.io/2012/06/15/The-true-power-of-regular-expressions.html), however the fact that you can, *does not mean that you should*.

## Other options?

We can use *parser generators*!

- [PLY](http://www.dabeaz.com/ply/): pythonic `lex/yacc`
- [ANTLR](http://www.antlr.org)

Parser generators are *fast*, but:
- are basically external tools,
- require separate grammar definition files

## Any other solution?

**YES**: combinatory parsing with [pyparsing](http://pyparsing.wikispaces.com/)

Idea: build a complex parsers by combining simple ones;
- everything is in the python source code
- very functional (in a FP sense) style 

## Let's start by example

In [2]:
from pyparsing import *

We will write a simple parser for positive integers.

PyParsing defines some strings of commonly used characters. In particular:

In [3]:
nums

'0123456789'

In [4]:
parsePosInt = Word(nums)

`Word` is a PyParsing object that parses sequence of characters, delimited by spaces. In order to perform actuall parsing we need to call a method of PyParsing object: 

In [5]:
parsePosInt.parseString("1234")

(['1234'], {})

When the parsing fails, exception is raised.

In [6]:
parsePosInt.parseString("foo")

ParseException: Expected W:(0123...) (at char 0), (line:1, col:1)

### Improving a little bit

String `000123` should not be parsed as integer.

In [7]:
parsePosInt.parseString('000123')

(['000123'], {})

Let's summon signature of Word constructor: `Word.__init__(initChars, bodyChars=None,...)`

In [8]:
parsePosInt = Word(nums[1:], nums)

In [9]:
parsePosInt.parseString("123")

(['123'], {})

In [10]:
parsePosInt.parseString("000123")

ParseException: Expected W:(1234...,0123...) (at char 0), (line:1, col:1)

### Adding an optional sign

We want to parse `-4` and `+10` as well.

In [11]:
parseSign = Literal('+') | Literal('-')

`Literal` is a PyParsing object that parses the exact string given in constructor. The `|` operator used on PyParsing objects returns another PyParsing object: `MatchFirst`:

In [12]:
type(parseSign)

pyparsing.MatchFirst

It parses either first or the second object.

In [13]:
parseSign.parseString('+')

(['+'], {})

In [14]:
parseSign.parseString('-')

(['-'], {})

Finally, we combine the `parsePosInt` and `parseSign` parsers:

In [15]:
parseInt = Optional(parseSign) + parsePosInt

`Optional`, as name suggests, *optionally* parses its argument. The `+` operator returns the `And` PyParsing object. It concatenates parsers.

In [16]:
parseInt.parseString('44')

(['44'], {})

In [17]:
parseInt.parseString('-11')

(['-', '11'], {})

In [18]:
parseInt.parseString('+12')

(['+', '12'], {})

Finally, we (might) want the result to be concatenated (e.g. to call `int` function on it). That's what the `Combine` object do.

In [19]:
parseInt = Combine(Optional(parseSign) + parsePosInt)

In [20]:
parseInt.parseString('44')

(['44'], {})

In [21]:
parseInt.parseString('-11')

(['-11'], {})

In [22]:
parseInt.parseString('+12')

(['+12'], {})

### PyParsing parsers are objects

Some of them define *primitive* parsers, like `Word`, some defines *combinations* (a kind of higher-order parsers) like `+`, `|` or `Optional`.

In [23]:
type(parsePosInt)

pyparsing.Word

In [24]:
(Word.__base__, Token.__base__)

(pyparsing.Token, pyparsing.ParserElement)

In [25]:
(And.__base__, ParseExpression.__base__)

(pyparsing.ParseExpression, pyparsing.ParserElement)

In [26]:
(Combine.__base__, TokenConverter.__base__, ParseElementEnhance.__base__)

(pyparsing.TokenConverter,
 pyparsing.ParseElementEnhance,
 pyparsing.ParserElement)

Every PyParsing object is a subclass of `ParserElement`.

## Git url parser

Let's recall what we need to parse: 

- `ssh://user@server/project.git`, 
- `https://example.com/gitproject.git`,
- `file:///opt/git/project.git`

### Remote URLs

We start with parsing protocol type. Remote and local protocols will be treated separately (locally we don't have host and username to parse).

In [27]:
parseRemoteProto = Literal('ssh://') | Literal('https://')

Syntax for hostname and username (at least in first approximation) is common.

In [28]:
ident = Word(alphanums, alphanums + "._-")

parseUserName = Combine(ident + Suppress(Literal("@")))
parseHost = Combine(ident + FollowedBy(Literal("/")))

We see two new combinators:

- `Suppress` matches given object but excludes it from returned result.
- `FollowedBy` matches given object but does not include it in returned result, nor advances pointer.

Now we combine this together and give *names* to different parts of parser object (`ParserElement.__call__` method is a shortcut for `ParserElement.setResultsName` method that sets the name):

In [29]:
parseRemote = parseRemoteProto('proto') + \
    Optional(parseUserName('user')) + \
    parseHost('host')

In [30]:
parseRemote.parseString('https://server/').asDict()

{'host': 'server', 'proto': 'https://'}

In [31]:
parseRemote.parseString('ssh://foo@server/').asDict()

{'host': 'server', 'proto': 'ssh://', 'user': 'foo'}

### Local URLs and the repo

Local protocol is simply:

In [32]:
parseLocal = Literal('file://')

And we assume that the rest of input is path of repository:

In [33]:
parsePath = restOfLine
parseGitURL = (parseRemote | parseLocal) + parsePath('repo')

In [34]:
parseGitURL.parseString('ssh://foo@server/project.git').asDict()

{'host': 'server', 'proto': 'ssh://', 'repo': '/project.git', 'user': 'foo'}

In [35]:
parseGitURL.parseString('https://server/project.git').asDict()

{'host': 'server', 'proto': 'https://', 'repo': '/project.git'}

In [36]:
parseGitURL.parseString('file:///opt/git/project.git').asDict()

{'repo': '/opt/git/project.git'}

## What we've learnt so far

- `pyparsing` parsers are objects; we use method `parseString` to perform actual parsing;
- objects can be combined together: `+` concatenates, `|` is logical *or*, `Optional` creates optional parser, etc.;
- we can give names to interesting expressions;
- testing is very easy -- we just verify the smallest compontents.

Time for more sophistacted combinators

## Parse one of many

In [37]:
parseFruit = (Literal('apple') | Literal('orange') | Literal('plum') 
              | Literal('pear'))

In [38]:
from functools import reduce
parseFruit = reduce(lambda a, b: a | b, 
                    [Literal('apple'), 'orange', 'plum', 'pear'])

In [39]:
parseFruit.parseString('apple')

(['apple'], {})

In [40]:
parseFruit.parseString('carrot')

ParseException: Expected {"apple" | "orange" | "plum" | "pear"} (at char 0), (line:1, col:1)

Of course this is so common use case that there is a function for that:

In [41]:
parseFruit = oneOf(["apple", "orange", "plum", "pear"])

In [42]:
parseFruit.parseString("plum")

(['plum'], {})

## Repeating parsers

Say that we want to parse a space delimited list of fruits (of unknown lenght):

In [43]:
fruits = Group(reduce(lambda a, b: a + Optional(b), [parseFruit]*10))('fruits')

`Group` takes all matched results and combine them into the list (do not confuse with `Combine` which concatenates results).

In [44]:
fruits.parseString("apple").asDict()

{'fruits': ['apple']}

In [45]:
fruits.parseString("apple plum").asDict()

{'fruits': ['apple', 'plum']}

However, there is a separate object that does such thing (and do it better):

In [46]:
fruits = OneOrMore(parseFruit)

In [47]:
fruits.parseString("apple pear")

(['apple', 'pear'], {})

In [48]:
fruits.parseString("plum, pear")

(['plum'], {})

In [49]:
fruits.parseString("plum pear carrot")

(['plum', 'pear'], {})

`OneOrMore` is like `+` modifier in regex. There is also something like regex `*` modifier: 
the `ZeroOrMore`. Note that it is always successful:

In [50]:
parseVege = oneOf(["carrot", "cucumber", "onion", "potato"])

In [51]:
fruitOrVege = ZeroOrMore(parseFruit) | ZeroOrMore(parseVege)

In [52]:
fruitOrVege.parseString("onion")

([], {})

We need to be careful with `|` combinator. Alternatively, `^` is like `|` but returns the longest of parsed result:

In [53]:
fruitOrVege = ZeroOrMore(parseFruit) ^ ZeroOrMore(parseVege)

In [54]:
fruitOrVege.parseString("onion")

(['onion'], {})

## Parsing lists

Lists usually have some custom delimiter.

In [55]:
def parseList(elem, delim=","): return elem + ZeroOrMore(Suppress(delim) + elem)

In [56]:
parseList(parseFruit).parseString("apple, plum, orange")

(['apple', 'plum', 'orange'], {})

In [57]:
parseList(parseFruit, delim=";").parseString("apple; plum; orange")

(['apple', 'plum', 'orange'], {})

Again, there is already a function that [does that](https://pythonhosted.org/pyparsing/pyparsing-module.html#delimitedList):

In [58]:
delimitedList(Word(hexnums), delim=':', combine=True)\
    .parseString("AA:BB:CC:DD:EE")

(['AA:BB:CC:DD:EE'], {})

## Parsing dictionaries...

...right into Python dictionaries.

In [59]:
shoppingList = dictOf(parseFruit + Suppress(":"), parsePosInt)

In [60]:
shoppingList.parseString('apple: 3 plum: 25 orange: 5').asDict()

{'apple': '3', 'orange': '5', 'plum': '25'}

To have different value parsers for different keys we need to write our own function. But it's not hard at all!

In [61]:
def safeDictOf(grammar, sep=':'):
    """ grammar is a list of pairs: the first value is a parser for key,
    the second is a parser for corresponding value"""
    def elem(g):
        (k, v) = g
        return Group(k + Suppress(sep) + v)

    opts = reduce(lambda acc, el: acc | el,
                  map(elem, grammar))
    return Dict(ZeroOrMore(opts))

In [62]:
funnyShoppingList = safeDictOf([(parseFruit, parsePosInt), 
                                (parseVege, Word(hexnums))])

In [63]:
funnyShoppingList.parseString("apple: 10 carrot: AE").asDict()

{'apple': '10', 'carrot': 'AE'}

In [64]:
funnyShoppingList.parseString("apple: 10 plum: AE").asDict()

{'apple': '10'}

### Parsing to the end

requires just adding a parser object that matches end of string:

In [65]:
(funnyShoppingList + StringEnd()).parseString("apple: 10 plum: AE").asDict()

ParseException: Expected end of text (at char 10), (line:1, col:11)

## Parse actions

It is possible to perform certain action when parsing object matches, like:

- converting numbers
- setting boolean values
- creating some complex objects

In [66]:
def isKeyword(kwd):
    return Keyword(kwd).setParseAction(lambda s, loc, toks: True)

The `setParseAction` methods assigns given function to successful parse of object. Arguments of that function are:

- `s` the original string being parsed
- `loc` the location of the matching substring
- `toks` a list of the matched tokens

In [67]:
isKeyword("verbose")("vflag").parseString('verbose').asDict()

{'vflag': True}

### Converting numbers

It is another typical use for parse actions. Let's use previously defined parser for integers:

In [68]:
integer = parseInt.copy().setParseAction(lambda s, l, t: int(t[0]))

In [69]:
integer('x').parseString('1024').asDict()

{'x': 1024}

Parsing floating point number

In [70]:
# parse decimal strings, e.g. 1.2, -5.44, etc
decimalStr = Regex('[-+]?\d+(\.\d*)?') | Regex('[-+]?\.\d+')
doubleStr = Combine(decimalStr + (Literal('e') | Literal('E')) + parseInt)\
            | Literal('inf') | Literal('-inf') | Literal('+inf') \
            | decimalStr
double = doubleStr.copy().setParseAction(lambda s, l, t: [float(t[0])])

The `Regex` object, as name suggest, is PyParsing object matching given regex.

In [71]:
double.parseString('3.14')

([3.14], {})

In [72]:
double.parseString('1.2e-12')

([1.2e-12], {})

### Evaluating simple arithmetic operations

We can use parse action to further combine results of previous parse actions:

In [73]:
addition = (double + Suppress(Keyword('+')) + double)\
           .setParseAction(lambda s, l, t: t[0] + t[1])
multiplication = (double + Suppress(Literal('*')) + double)\
                 .setParseAction(lambda s, l, t: t[0] * t[1])

In [74]:
addition.parseString("2 + 3")

([5.0], {})

In [75]:
multiplication.parseString("4*3")

([12.0], {})

### Simple calculator: recursive grammars

It is also possible to define grammar in a *recursive* way. So that we can write a very simple calculator of arithmetic expressions:

In [76]:
addition = Forward()
multiplication = Forward()

This is a "promise" that we will define `addition` and `multiplication` parsers. Recursive grammar has to be defined in such way. Next, we define a groupped expression:

In [77]:
group = Suppress('(') + (addition | multiplication) + Suppress(')')

Finally we define addition or multiplication of either numbers or grouped expressions.

In [78]:
addition << ((double | group) + Suppress(Keyword('+')) + (double | group))\
           .setParseAction(lambda s, l, t: t[0] + t[1])
multiplication << \
    ((double | group) + Suppress(Literal('*')) + (double | group))\
    .setParseAction(lambda s, l, t: t[0] * t[1])
expr = addition | multiplication

In [79]:
expr.parseString("2 + 5")

([7.0], {})

In [80]:
expr.parseString("3 * 7")

([21.0], {})

In [81]:
expr.parseString("2 + (3 * 2)")

([8.0], {})

In [82]:
expr.parseString("((2 + 5) * 5) + (3 * (2 + 3))")

([50.0], {})

In [83]:
eval("(2 + 5) * 5 + 3 * (2 + 3)")

50

## Real life use

In <a href="http://zettafox.com" style="text-decoration: none"><b><font face="Book Antiqua, serif" color="#7f7f7f"><span style="font-size:120%;line-height:107%">zetta</span></font></b><b><font color="#ed7d31"><font face="Book Antiqua, serif"><span style="font-size:120%;line-height:107%">fox </span></font></font></b></a> we used to parse user input in two CLI tools and one custom jupyter kernel:

<img width="600px" src="ade-pic1.png"/>

and to serialize *rules* (predicates on dataframe) in human readable (and editable) format: 

<img width="700px" src="rs-pic2.png"/>

### Benefits

- human-friendly grammar in CLI tools
- possible to verify input on parsing level
- easy to extend, modify
- very modular: big parts of parsers can be shared.
- easy to test
- human readable serialization resulted cleaner and nicer unittests

## Thank you

Further reading:

- [PyParsing API](https://pythonhosted.org/pyparsing/),
- Haskell's [Parsec](https://hackage.haskell.org/package/parsec) library.
